<a href="https://colab.research.google.com/github/Zaicevvv/GoIT-DSandML-Private/blob/main/HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Завантажуємо дані з CSV-файлів, витягуємо часові ознаки з акселерометра та розділяємо дані на навчальний та тестовий набори. Також нормалізуємо дані за допомогою StandardScaler.

In [8]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Папка, де зберігаються CSV-файли
data_folder = Path("/content/drive/MyDrive/data")
# Створимо порожні списки для даних та міток класів
X = []
y = []

# Функція для витягнення часових ознак з акселерометра
def extract_features(data):
    features = []
    # Додамо середнє значення для кожного стовпця
    features.extend(np.mean(data, axis=0))
    # Додамо середнє квадратичне відхилення для кожного стовпця
    features.extend(np.std(data, axis=0))
    # Додамо медіану для кожного стовпця
    features.extend(np.median(data, axis=0))
    # Додамо мінімальне та максимальне значення для кожного стовпця
    features.extend(np.min(data, axis=0))
    features.extend(np.max(data, axis=0))
    return features

# Пройдемось по всім папкам та файлам, зчитаємо дані та витягнемо ознаки
for folder in data_folder.iterdir():
    for csv_file in folder.iterdir():
        data = pd.read_csv(csv_file)

        # Витягнемо ознаки та додамо мітку класу
        features = extract_features(data.values)
        X.append(features)
        y.append(folder.name)

# Перетворимо списки у numpy масиви
X = np.array(X)
y = np.array(y)

# Розділимо дані на навчальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормалізуємо дані
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Перейдемо до навчання моделей SVM та випадкового лісу.

In [9]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Навчання моделі SVM
svm_model = SVC(kernel='linear').fit(X_train_scaled, y_train)

# Навчання моделі випадкового лісу
rf_model = RandomForestClassifier(n_estimators=100, random_state=42).fit(X_train_scaled, y_train)

# Оцінка результатів
svm_predictions = svm_model.predict(X_test_scaled)
rf_predictions = rf_model.predict(X_test_scaled)

# Отримання звіту про класифікацію
svm_report = classification_report(y_test, svm_predictions)
rf_report = classification_report(y_test, rf_predictions)

print("SVM classification report:")
print(svm_report)

print("\nRandom Forest classification report:")
print(rf_report)

SVM classification report:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       208
     running       1.00      1.00      1.00       701
      stairs       0.95      0.60      0.73        30
     walking       0.97      1.00      0.98       354

    accuracy                           0.99      1293
   macro avg       0.98      0.90      0.93      1293
weighted avg       0.99      0.99      0.99      1293


Random Forest classification report:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       208
     running       1.00      1.00      1.00       701
      stairs       1.00      1.00      1.00        30
     walking       1.00      1.00      1.00       354

    accuracy                           1.00      1293
   macro avg       1.00      1.00      1.00      1293
weighted avg       1.00      1.00      1.00      1293



У підсумку можна сказати, що випадковий ліс показав ідеальні результати на тестових даних, на відміну від SVM.